In [39]:
# Tutorial @ http://jrmeyer.github.io/tutorial/2016/02/01/TensorFlow-Tutorial.html

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tarfile
import os
import time


In [40]:
"""Import Data"""
dir_ = 'data_dir'
tar_file = 'data.tar.gz'


def csv_to_numpy_array(file_path, delimiter):
    """csv to numpy array"""
    return np.genfromtxt(file_path, delimiter=delimiter, dtype=None)


def import_data():
    """importing data"""
    if dir_ not in os.listdir(os.getcwd() + '/007_NLP/HamOrSpam'):
        # Untar directory of data if we haven't already
        tar_object = tarfile.open(os.getcwd() + '/007_NLP/HamOrSpam/' + tar_file)
        tar_object.extractall()
        tar_object.close()
        print('Extracted tar file to current directory')
        print('Extracted again')
    else:
        pass
    print("loading training data")
    train_x = csv_to_numpy_array("data/trainX.csv", delimiter="\t")
    train_y = csv_to_numpy_array("data/trainY.csv", delimiter="\t")
    print("loading test data")
    test_x = csv_to_numpy_array("data/testX.csv", delimiter="\t")
    test_y = csv_to_numpy_array("data/testY.csv", delimiter="\t")
    return train_x, train_y, test_x, test_y


# extracting data
train_x, train_y, test_x, test_y = import_data()

# params
num_features = train_x.shape[1]  # num of features(columns) - 2955
num_labels = train_y.shape[1]  # num of labels(columns) - 2
epochs = 25000
# a smarter learning rate for GradientDescent
learning_rate = tf.train.exponential_decay(learning_rate=.0008,
                                           global_step=1,
                                           decay_steps=train_x.shape[0],
                                           decay_rate=.95,
                                           staircase=True)

# placeholders
# X = X-matrix / feature-matrix / data-matrix... It's a tensor to hold our email
# data. 'None' here means that we can hold any number of emails
X = tf.placeholder(tf.float32, shape=[None, num_features])

# Y = Y-matrix / label-matrix / labels... This will be our correct answers
# matrix. Every row has either [1,0] for SPAM or [0,1] for HAM. 'None' here 
# means that we can hold any number of emails
Y = tf.placeholder(tf.float32, shape=[None, num_labels])

# variables
# Values are randomly sampled from a Gaussian with a standard deviation of:
#     sqrt(6 / (numInputNodes + numOutputNodes + 1))

weights = tf.Variable(tf.random_normal([num_features, num_labels],
                                       mean=0,
                                       stddev=(np.sqrt(6 / num_features + num_labels + 1)),
                                       name='weights'))

bias = tf.Variable(tf.random_normal([1, num_labels],
                                    mean=0,
                                    stddev=np.sqrt(6 / num_labels + num_labels + 1)),
                   name='bias')

# init vars
init_vars = tf.global_variables_initializer()

# logistic regression
# (i.e. ŷ =sig(WX+b)y^=sig(WX+b))

# feed forward - prediction algorithm
apply_weight_op = tf.matmul(X, weights, name='apply_weights')
add_bias_op = tf.add(apply_weight_op, bias, name='add_bias')
activation_op = tf.nn.sigmoid(add_bias_op, name='activation')

# cost/loss operation
# MSE -{ (1/2)summation(i = 1 .... N)[(Ypred-i - Yactual-i)^2]}
cost_op = tf.nn.l2_loss(activation_op - Y, name='MSE')

# optimization op
optimization_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_op)

# visualization
epoch_vals = []
accuracy_vals = []
cost_vals = []
# turn on interactive plotting
plt.ion()

# create main, super plot
fig = plt.figure()
# Create two subplots on their own axes and give titles
ax1 = plt.subplot("211")
ax1.set_title("TRAINING ACCURACY", fontsize=18)
ax2 = plt.subplot("212")
ax2.set_title("TRAINING COST", fontsize=18)
plt.tight_layout()

with tf.Session() as sess:
    sess.run(init_vars)
    # Ops for vizualization
    # argmax(activation_op, 1) gives the label our model thought was most likely
    # argmax(Y, 1) is the correct label
    correct_predictions_OP = tf.equal(tf.argmax(activation_op, 1), tf.argmax(Y, 1))
    # False is 0 and True is 1, what was our average?
    accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))
    # Summary op for regression output
    activation_summary_OP = tf.summary.histogram("output", activation_op)
    # Summary op for accuracy
    accuracy_summary_OP = tf.summary.scalar("accuracy", accuracy_OP)
    # Summary op for cost
    cost_summary_OP = tf.summary.scalar("cost", cost_op)
    # Summary ops to check how variables (W, b) are updating after each iteration
    weightSummary = tf.summary.histogram("weights", weights.eval(session=sess))
    biasSummary = tf.summary.histogram("biases", bias.eval(session=sess))
    # Merge all summaries
    # Merge all summaries
    all_summary_OPS = tf.summary.merge_all()
    # Summary writer
    writer = tf.summary.FileWriter("summary_logs", sess.graph)
    # Initialize reporting variables
    cost = 0
    diff = 1

    # Training epochs
    for i in range(epochs):
        if i > 1 and diff < .0001:
            print("change in cost %g; convergence." % diff)
            break
        else:
            # training
            step = sess.run(optimization_op, feed_dict={X: train_x, Y: train_y})

            # Report occasional stats
        if i % 10 == 0:
            # Add epoch to epoch_values
            epoch_vals.append(i)
            # Generate accuracy stats on test data
            summary_results, train_accuracy, newCost = sess.run(
                [all_summary_OPS, accuracy_OP, cost_op],
                feed_dict={X: train_x, Y: train_y}
            )
            # Add accuracy to live graphing variable
            accuracy_vals.append(train_accuracy)
            # Add cost to live graphing variable
            cost_vals.append(newCost)
            # Write summary stats to writer
            writer.add_summary(summary_results, i)
            # Re-assign values for variables
            diff = abs(newCost - cost)
            cost = newCost

            #generate print statements
            print("step %d, training accuracy %g" % (i, train_accuracy))
            print("step %d, cost %g" % (i, newCost))
            print("step %d, change in cost %g" % (i, diff))

            # Plot progress to our two subplots
            accuracyLine, = ax1.plot(epoch_vals, accuracy_vals)
            costLine, = ax2.plot(epoch_vals, cost_vals)
            fig.canvas.draw()
            time.sleep(1)

    # How well do we perform on held-out test data?
    print("final accuracy on test set: %s" % str(sess.run(accuracy_OP,
                                                          feed_dict={X: test_x,
                                                                     Y: test_y})))

    # Create Saver
    saver = tf.train.Saver()
    # Save variables to .ckpt file
    saver.save(sess, "trained_variables.ckpt")
    print('all done!')


Extracted tar file to current directory
Extracted again
loading training data


loading test data


step 0, training accuracy 0.530955
step 0, cost 288.369
step 0, change in cost 288.369


step 10, training accuracy 0.530955
step 10, cost 266.656
step 10, change in cost 21.7139


step 20, training accuracy 0.528856
step 20, cost 253.354
step 20, change in cost 13.3017


...

step 3950, training accuracy 0.763903
step 3950, cost 211.974
step 3950, change in cost 0.0666809


step 3960, training accuracy 0.764953
step 3960, cost 211.907
step 3960, change in cost 0.0664978


step 3970, training accuracy 0.764953
step 3970, cost 211.841
step 3970, change in cost 0.0665283


step 3980, training accuracy 0.764953
step 3980, cost 211.774
step 3980, change in cost 0.0664368


step 3990, training accuracy 0.764953
step 3990, cost 211.708
step 3990, change in cost 0.0663757
